# 1. Some Utils Tools

In [1]:
import sys
import pandas as pd
rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')


from atom import Atom
from dataset import batch1, batch2

batch = batch1 
# this dictionary is used to deal with the annotated entities in texts.
fLabel = batch['dataAnno']['fLabel'];            print(fLabel)


def strQ2B(ustring):
    rstring = ''
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):
            inside_code -= 65248
        # 2: unichr; 3: chr
        rstring += chr(inside_code)
    return rstring

def cleanSpace(text):
    #return text.replace("\t"," ").replace(" ","&nbsp;").replace('\n', '')
    '''
    return text.strip()
    '''
    for i in ' \t\r\x0b\x0c':
        text = text.replace(i, '@')
    text = text.strip()
    text =  text.replace('\n', '@')
    return text 
    
def segmentStr(text):
    strLen = len(text)
    atoms = []
    for i in range(strLen):
        atoms.append(text[i])
    return atoms

{'症状和体征': 'Sy', '身体部位': 'Bo', '检查和检验': 'Ch', '治疗': 'Tr', '疾病和诊断': 'Si'}


# 2.  `__init__`

In [2]:
class ChineseClinicalText(object):
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None, sep = '\t'):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)

# 3. Atom tags


---


Have a look at the input files.


TODO


---

In [3]:
batch1SampleOrig = 'demo/Text/batch1/orig.txt'
batch1SampleAnno = 'demo/Text/batch1/anno.txt'
batch2SampleOrig = 'demo/Text/batch2/orig.txt'
batch2SampleAnno = 'demo/Text/batch2/anno.txt'

with open(batch1SampleOrig, 'r') as f:
    text = f.read()  
text

'中年男性，49岁，主因：腹痛、腹胀2个月，停止排便3天，停止排气2天。于2016-3-30入院。\n\n\n\n'

In [4]:
cct = ChineseClinicalText(text)
cct._text
length = len(cct._text)
print(strQ2B(text)[:length] == cct._text); print(cct.atomsBaseList)

True
['中', '年', '男', '性', ',', '4', '9', '岁', ',', '主', '因', ':', '腹', '痛', '、', '腹', '胀', '2', '个', '月', ',', '停', '止', '排', '便', '3', '天', ',', '停', '止', '排', '气', '2', '天', '。', '于', '2', '0', '1', '6', '-', '3', '-', '3', '0', '入', '院', '。']


In [5]:
cct.setAtomTags()
for a in cct.atoms[:10]:
    print(a)

中	CHN	OTHER	丨
年	CHN	OTHER	干
男	CHN	OTHER	田
性	CHN	OTHER	忄
,	PUNC	OTHER	-
4	NUM	OTHER	-
9	NUM	OTHER	-
岁	CHN	OTHER	山
,	PUNC	OTHER	-
主	CHN	OTHER	丶


# 4. POS Tags

In [7]:
from jieba import posseg as ppseg
segs = list(ppseg.cut(cct._text)); print(segs[:10])
pair = segs[0]
pair.word

[pair('中年', 't'), pair('男性', 'n'), pair(',', 'x'), pair('49', 'm'), pair('岁', 'm'), pair(',', 'x'), pair('主因', 'n'), pair(':', 'x'), pair('腹痛', 'n'), pair('、', 'x')]


'中年'

In [8]:
for i in range(len(segs)):
    pair = segs[i]
    start = sum(len(p.word) for p in segs[:i])
    end = sum(len(p.word) for p in segs[:i+1]) -1
    pair.indeces = [start, end]
    # print( text[start: end +1], '\t',pair.flag, '\t',  pair.indeces)
    
## Test Results
for p in segs:
    print(p.word,'\t' , p.flag,'\t', p.indeces)

中年 	 t 	 [0, 1]
男性 	 n 	 [2, 3]
, 	 x 	 [4, 4]
49 	 m 	 [5, 6]
岁 	 m 	 [7, 7]
, 	 x 	 [8, 8]
主因 	 n 	 [9, 10]
: 	 x 	 [11, 11]
腹痛 	 n 	 [12, 13]
、 	 x 	 [14, 14]
腹胀 	 v 	 [15, 16]
2 	 m 	 [17, 17]
个 	 m 	 [18, 18]
月 	 m 	 [19, 19]
, 	 x 	 [20, 20]
停止 	 v 	 [21, 22]
排便 	 v 	 [23, 24]
3 	 m 	 [25, 25]
天 	 n 	 [26, 26]
, 	 x 	 [27, 27]
停止 	 v 	 [28, 29]
排气 	 n 	 [30, 31]
2 	 m 	 [32, 32]
天 	 n 	 [33, 33]
。 	 x 	 [34, 34]
于 	 p 	 [35, 35]
2016 	 m 	 [36, 39]
- 	 x 	 [40, 40]
3 	 x 	 [41, 41]
- 	 x 	 [42, 42]
30 	 m 	 [43, 44]
入院 	 n 	 [45, 46]
。 	 x 	 [47, 47]


In [9]:
POSTagsList = []
for p in segs:
    word = p.word
    for i in range(len(p.word)):
        if i == 0:
            POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
        else:
            POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
POSTagsList[:10]

[[0, '中', 't-B'],
 [1, '年', 't-I'],
 [2, '男', 'n-B'],
 [3, '性', 'n-I'],
 [4, ',', 'x-B'],
 [5, '4', 'm-B'],
 [6, '9', 'm-I'],
 [7, '岁', 'm-B'],
 [8, ',', 'x-B'],
 [9, '主', 'n-B']]

In [10]:
def getPOSTagsList(text):
    segs = list(ppseg.cut(text))
    
    for i in range(len(segs)):
        pair = segs[i]
        start = sum(len(p.word) for p in segs[:i])
        end = sum(len(p.word) for p in segs[:i+1]) -1
        pair.indeces = [start, end]
        
    POSTagsList = []
    for p in segs:
        word = p.word
        for i in range(len(p.word)):
            if i == 0:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
            else:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
                
    return POSTagsList

In [11]:
getPOSTagsList(text)[:10]

[[0, '中', 't-B'],
 [1, '年', 't-I'],
 [2, '男', 'n-B'],
 [3, '性', 'n-I'],
 [4, '，', 'x-B'],
 [5, '4', 'm-B'],
 [6, '9', 'm-I'],
 [7, '岁', 'm-B'],
 [8, '，', 'x-B'],
 [9, '主', 'n-B']]

In [12]:
import pandas as pd

class ChineseClinicalText(object):
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
    
    # Here we also use the tools of Pandas.
    def updateDF(self):
        #update the attribute for pandas PF form sentence
        DF = pd.DataFrame([i.to_dict() for i in self.atoms])
        cols = ['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags
        self.dfFormat = DF[cols] # Here is to have a good order
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


---

### Test Code

**To show how `.updateDF` and `.toTextFile` work.**

---

In [13]:
cct = ChineseClinicalText(text); print(cct._text)
text

中年男性,49岁,主因:腹痛、腹胀2个月,停止排便3天,停止排气2天。于2016-3-30入院。


'中年男性，49岁，主因：腹痛、腹胀2个月，停止排便3天，停止排气2天。于2016-3-30入院。\n\n\n\n'

In [14]:
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
a = cct.atoms[0]
a.to_dict()  # here is to the pd.DataFrame

{'POSTag': 't-B',
 '_atom': '中',
 'basicTag': 'CHN',
 'dictTag': 'OTHER',
 'index': 0,
 'radical': '丨'}

In [15]:
DF = pd.DataFrame([i.to_dict() for i in cct.atoms])
a = DF[['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag']]

cols = ['_atom', 'basicTag', 'dictTag', 'POSTag']

a.to_csv('demo/Text/toTextFile.txt', 
         sep = '\t', 
         encoding = 'utf-8', 
         header = False,
         index = False, 
         columns = cols)
a.head()

,_atom,basicTag,dictTag,radical,POSTag
0,中,CHN,OTHER,丨,t-B
1,年,CHN,OTHER,干,t-I
2,男,CHN,OTHER,田,n-B
3,性,CHN,OTHER,忄,n-I
4,",",PUNC,OTHER,-,x-B


# 5. Deal with the CRF Result

---

### `getLearnedETag` and `.corpResult`

---

This time only `ETags` will be considered.

The `ETags` are learned from trained models.

They are used to evaluate the models' performances

or are processed to be the `tagger`'s results.

Here we show a tagged file: `'demo/Text/batch1/orig_tagged.txt'`



In [16]:
outputPath = 'demo/Text/batch1/orig_tagged.txt'
L = []
with open(outputPath, 'r') as f:
    for line in f.readlines():
        if '# 0' in line[:3] or '# 1' in f.readlines():
            L.append(line)
            
L

['# 0.427237\n']

In [17]:
result = pd.read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result.iloc[:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,中,CHN,OTHER,t-B,O/0.825244,Bo-B/0.000241,Bo-I/0.000977,Ch-B/0.000126,Ch-I/0.001453,DT-B/0.004608,DT-I/0.001729,O/0.825244,Si-B/0.148639,Si-I/0.013709,Sy-B/0.000620,Sy-I/0.000649,Tr-B/0.000644,Tr-I/0.000572,unct-B/0.000395,unct-I/0.000394
1,年,CHN,OTHER,t-I,O/0.837903,Bo-B/0.000004,Bo-I/0.000212,Ch-B/0.000003,Ch-I/0.000276,DT-B/0.000017,DT-I/0.001286,O/0.837903,Si-B/0.000130,Si-I/0.158759,Sy-B/0.000036,Sy-I/0.000867,Tr-B/0.000028,Tr-I/0.000333,unct-B/0.000011,unct-I/0.000134
2,男,CHN,OTHER,n-B,O/0.840708,Bo-B/0.000065,Bo-I/0.000079,Ch-B/0.000088,Ch-I/0.000065,DT-B/0.000008,DT-I/0.000569,O/0.840708,Si-B/0.000097,Si-I/0.156674,Sy-B/0.000208,Sy-I/0.000845,Tr-B/0.000096,Tr-I/0.000200,unct-B/0.000010,unct-I/0.000289
3,性,CHN,OTHER,n-I,O/0.845097,Bo-B/0.000003,Bo-I/0.000155,Ch-B/0.000007,Ch-I/0.000164,DT-B/0.000008,DT-I/0.000523,O/0.845097,Si-B/0.000002,Si-I/0.151787,Sy-B/0.000003,Sy-I/0.001120,Tr-B/0.000011,Tr-I/0.000256,unct-B/0.000004,unct-I/0.000860
4,",",PUNC,OTHER,x-B,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,DT-B/0.000000,DT-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000001,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000
5,4,NUM,OTHER,m-B,O/0.999993,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000004,Ch-I/0.000000,DT-B/0.000001,DT-I/0.000000,O/0.999993,Si-B/0.000001,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000
6,9,NUM,OTHER,m-I,O/0.999988,Bo-B/0.000001,Bo-I/0.000000,Ch-B/0.000001,Ch-I/0.000000,DT-B/0.000001,DT-I/0.000000,O/0.999988,Si-B/0.000004,Si-I/0.000001,Sy-B/0.000002,Sy-I/0.000000,Tr-B/0.000001,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000
7,岁,CHN,OTHER,m-B,O/0.999811,Bo-B/0.000010,Bo-I/0.000005,Ch-B/0.000039,Ch-I/0.000005,DT-B/0.000014,DT-I/0.000014,O/0.999811,Si-B/0.000018,Si-I/0.000019,Sy-B/0.000016,Sy-I/0.000005,Tr-B/0.000030,Tr-I/0.000004,unct-B/0.000004,unct-I/0.000006
8,",",PUNC,OTHER,x-B,O/1.000000,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,DT-B/0.000000,DT-I/0.000000,O/1.000000,Si-B/0.000000,Si-I/0.000000,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000
9,主,CHN,OTHER,n-B,O/0.996299,Bo-B/0.001163,Bo-I/0.000005,Ch-B/0.000523,Ch-I/0.000003,DT-B/0.000032,DT-I/0.000005,O/0.996299,Si-B/0.000149,Si-I/0.000002,Sy-B/0.001469,Sy-I/0.000001,Tr-B/0.000298,Tr-I/0.000003,unct-B/0.000045,unct-I/0.000003


In [18]:
print(cols)
result.columns = cols + ['Result'] + list(range((len(result.columns)-len(cols) -1)))
result.head()

['_atom', 'basicTag', 'dictTag', 'POSTag']


,_atom,basicTag,dictTag,POSTag,Result,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,中,CHN,OTHER,t-B,O/0.825244,Bo-B/0.000241,Bo-I/0.000977,Ch-B/0.000126,Ch-I/0.001453,DT-B/0.004608,DT-I/0.001729,O/0.825244,Si-B/0.148639,Si-I/0.013709,Sy-B/0.000620,Sy-I/0.000649,Tr-B/0.000644,Tr-I/0.000572,unct-B/0.000395,unct-I/0.000394
1,年,CHN,OTHER,t-I,O/0.837903,Bo-B/0.000004,Bo-I/0.000212,Ch-B/0.000003,Ch-I/0.000276,DT-B/0.000017,DT-I/0.001286,O/0.837903,Si-B/0.000130,Si-I/0.158759,Sy-B/0.000036,Sy-I/0.000867,Tr-B/0.000028,Tr-I/0.000333,unct-B/0.000011,unct-I/0.000134
2,男,CHN,OTHER,n-B,O/0.840708,Bo-B/0.000065,Bo-I/0.000079,Ch-B/0.000088,Ch-I/0.000065,DT-B/0.000008,DT-I/0.000569,O/0.840708,Si-B/0.000097,Si-I/0.156674,Sy-B/0.000208,Sy-I/0.000845,Tr-B/0.000096,Tr-I/0.000200,unct-B/0.000010,unct-I/0.000289
3,性,CHN,OTHER,n-I,O/0.845097,Bo-B/0.000003,Bo-I/0.000155,Ch-B/0.000007,Ch-I/0.000164,DT-B/0.000008,DT-I/0.000523,O/0.845097,Si-B/0.000002,Si-I/0.151787,Sy-B/0.000003,Sy-I/0.001120,Tr-B/0.000011,Tr-I/0.000256,unct-B/0.000004,unct-I/0.000860
4,",",PUNC,OTHER,x-B,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,DT-B/0.000000,DT-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000001,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000


In [19]:
def getLearnedETag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None

a = result['Result'].apply(getLearnedETag)
a[5:20]

5        O
6        O
7        O
8        O
9        O
10       O
11       O
12    Sy-B
13    Sy-I
14       O
15    Sy-B
16    Sy-I
17       O
18       O
19       O
Name: Result, dtype: object

In [20]:
print(cols)

result['LearnedETag'] = result['Result'].apply(getLearnedETag)
result = result[cols + ['LearnedETag']]
result.iloc[5:25]

['_atom', 'basicTag', 'dictTag', 'POSTag']


,_atom,basicTag,dictTag,POSTag,LearnedETag
5,4,NUM,OTHER,m-B,O
6,9,NUM,OTHER,m-I,O
7,岁,CHN,OTHER,m-B,O
8,",",PUNC,OTHER,x-B,O
9,主,CHN,OTHER,n-B,O
10,因,CHN,BREAK_UNIT,n-I,O
11,:,PUNC,OTHER,x-B,O
12,腹,CHN,PART_UNIT,n-B,Sy-B
13,痛,CHN,SYM_UNIT,n-I,Sy-I
14,、,PUNC,OTHER,x-B,O


In [21]:
newTag = 'LearnedETag'

for idx, row in result.iterrows():
    for a in cct.atoms:
        if a.index == idx and a._atom == row['_atom']:
            a.__setattr__(newTag, row[newTag])
print(idx, row)
a.to_dict()

47 _atom              。
basicTag        PUNC
dictTag        OTHER
POSTag           x-B
LearnedETag        O
Name: 47, dtype: object


{'LearnedETag': 'O',
 'POSTag': 'x-B',
 '_atom': '。',
 'basicTag': 'PUNC',
 'dictTag': 'OTHER',
 'index': 47,
 'radical': '-'}

In [22]:
import pandas as pd

def getLearnedETag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None

class ChineseClinicalText(object):
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        # for Element is POSTags
        # [index, char, POS-Tag]
        # [8, '民', 'n-I']
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
                    #print(a, '\t', a.POSTag)
    
    # Here we also use the tools of Pandas.
    def updateDF(self):
        #update the attribute for pandas PF form sentence
        DF = pd.DataFrame([i.to_dict() for i in self.atoms])
        cols = ['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags
        self.dfFormat = DF[cols] # Here is to have a good order
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


        
    def corpResult(self, result, newTag):
        # The requirements on result
        # it is a dataframe
        # exactly the same as the CCT
        # include the right and same index of CCT's atoms
        ###### newTag will only be 'LearnedETag'
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getLearnedETag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
                     

In [23]:
#=================
# TEST
#=================
text

'中年男性，49岁，主因：腹痛、腹胀2个月，停止排便3天，停止排气2天。于2016-3-30入院。\n\n\n\n'

In [24]:
cct = ChineseClinicalText(text)
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
cct.updateDF()
cct.dfFormat.head()

,_atom,basicTag,dictTag,radical,POSTag
0,中,CHN,OTHER,丨,t-B
1,年,CHN,OTHER,干,t-I
2,男,CHN,OTHER,田,n-B
3,性,CHN,OTHER,忄,n-I
4,",",PUNC,OTHER,-,x-B


In [25]:
Orig_Test_Path = 'demo/Text/batch1/orig_test.txt'
cct.toTextFile(Orig_Test_Path, cols = cols); print(cols)
print(cct.outputCols)

['_atom', 'basicTag', 'dictTag', 'POSTag']
['_atom', 'basicTag', 'dictTag', 'POSTag']


In [26]:
## New we take result as given
outputPath
outputPath = 'demo/Text/batch1/orig_tagged.txt'
result = pd.read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,中,CHN,OTHER,t-B,O/0.825244,Bo-B/0.000241,Bo-I/0.000977,Ch-B/0.000126,Ch-I/0.001453,DT-B/0.004608,DT-I/0.001729,O/0.825244,Si-B/0.148639,Si-I/0.013709,Sy-B/0.000620,Sy-I/0.000649,Tr-B/0.000644,Tr-I/0.000572,unct-B/0.000395,unct-I/0.000394
1,年,CHN,OTHER,t-I,O/0.837903,Bo-B/0.000004,Bo-I/0.000212,Ch-B/0.000003,Ch-I/0.000276,DT-B/0.000017,DT-I/0.001286,O/0.837903,Si-B/0.000130,Si-I/0.158759,Sy-B/0.000036,Sy-I/0.000867,Tr-B/0.000028,Tr-I/0.000333,unct-B/0.000011,unct-I/0.000134
2,男,CHN,OTHER,n-B,O/0.840708,Bo-B/0.000065,Bo-I/0.000079,Ch-B/0.000088,Ch-I/0.000065,DT-B/0.000008,DT-I/0.000569,O/0.840708,Si-B/0.000097,Si-I/0.156674,Sy-B/0.000208,Sy-I/0.000845,Tr-B/0.000096,Tr-I/0.000200,unct-B/0.000010,unct-I/0.000289
3,性,CHN,OTHER,n-I,O/0.845097,Bo-B/0.000003,Bo-I/0.000155,Ch-B/0.000007,Ch-I/0.000164,DT-B/0.000008,DT-I/0.000523,O/0.845097,Si-B/0.000002,Si-I/0.151787,Sy-B/0.000003,Sy-I/0.001120,Tr-B/0.000011,Tr-I/0.000256,unct-B/0.000004,unct-I/0.000860
4,",",PUNC,OTHER,x-B,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,DT-B/0.000000,DT-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000001,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000


In [27]:
cct.corpResult(result, 'LearnedETag')
cct.updateDF()
cct.dfFormat.head()

,_atom,basicTag,dictTag,radical,POSTag,LearnedETag
0,中,CHN,OTHER,丨,t-B,O
1,年,CHN,OTHER,干,t-I,O
2,男,CHN,OTHER,田,n-B,O
3,性,CHN,OTHER,忄,n-I,O
4,",",PUNC,OTHER,-,x-B,O


In [28]:
print(cct.outputCols)
outputPath = 'demo/Text/batch1/toTextFile.txt'
cct.toTextFile(outputPath, cols = ['_atom', 'basicTag', 'LearnedETag'])
print(cct.outputCols)

['_atom', 'basicTag', 'dictTag', 'POSTag']
['_atom', 'basicTag', 'LearnedETag']


# 6. Deal With Learned Entities

In [29]:
taggedAtoms = [a for a in cct.atoms if a.LearnedETag != 'O']
startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].LearnedETag[-2:] == '-B']
startIdx.append(len(taggedAtoms))
startIdx

[0, 2, 4]

In [30]:
for i in range(len(startIdx)-1):
    entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
    startA, endA = entityAtom[0], entityAtom[-1]
    print([''.join(ea._atom for ea in entityAtom),startA.index, endA.index, startA.LearnedETag.split('-')[0]] )

['腹痛', 12, 13, 'Sy']
['腹胀', 15, 16, 'Sy']


In [31]:
import pandas as pd

def getLearnedETag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None

class ChineseClinicalText(object):
    
    def __init__(self, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
    

    def updateDF(self):
        #update the attribute for pandas PF form sentence
        DF = pd.DataFrame([i.to_dict() for i in self.atoms])
        cols = ['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags
        self.dfFormat = DF[cols] # Here is to have a good order
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


        
    def corpResult(self, result, newTag):
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getLearnedETag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
                     
         
    ############ NEW ###########
    def getLearnedEntities(self, eType):
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        return entitiesList

text

'中年男性，49岁，主因：腹痛、腹胀2个月，停止排便3天，停止排气2天。于2016-3-30入院。\n\n\n\n'

In [32]:
# text 
cct = ChineseClinicalText(text)
cct.setAtomTags()
cct.getPOSTags()
cct.setPOSTags()
cct.updateDF()
cols = ['_atom', 'basicTag', 'dictTag', 'POSTag'] # because of 1abdp model
Orig_Test_Path = 'demo/Text/batch1/orig_test.txt'
cct.toTextFile(Orig_Test_Path, cols = cols); print(cols)

outputPath = 'demo/Text/batch1/orig_tagged.txt'
result = pd.read_csv(outputPath, sep = '\t', comment = '#', header= None, skip_blank_lines= False)
cct.corpResult(result, 'LearnedETag')

cct.updateDF()

print(cct.getLearnedEntities('LearnedETag'))

cct.dfFormat.head()

['_atom', 'basicTag', 'dictTag', 'POSTag']
[['腹痛', 12, 13, 'Sy'], ['腹胀', 15, 16, 'Sy']]


,_atom,basicTag,dictTag,radical,POSTag,LearnedETag
0,中,CHN,OTHER,丨,t-B,O
1,年,CHN,OTHER,干,t-I,O
2,男,CHN,OTHER,田,n-B,O
3,性,CHN,OTHER,忄,n-I,O
4,",",PUNC,OTHER,-,x-B,O


# 7. Annotated Entities

In [33]:
batch = batch1

fLabel = batch['dataAnno']['fLabel']
fLabel

{'检查和检验': 'Ch', '治疗': 'Tr', '疾病和诊断': 'Si', '症状和体征': 'Sy', '身体部位': 'Bo'}

In [34]:
annotedEntities = [];                      print(batch1SampleAnno)
with open(batch1SampleAnno, 'r') as f:
    for l in f.readlines():
        entity = strQ2B(l).replace('\n','').split('\t')
        entity[-1] = fLabel[entity[-1]]
        entity[1], entity[2] = int(entity[1]), int(entity[2])
        annotedEntities.append(entity)
        
annotedEntities

demo/Text/batch1/anno.txt


[['腹痛', 12, 13, 'Sy'],
 ['腹胀', 15, 16, 'Sy'],
 ['排便', 23, 24, 'Sy'],
 ['排气', 30, 31, 'Sy']]

In [35]:
for e in annotedEntities:
    e[-1] = 'R'
annotedEntities

[['腹痛', 12, 13, 'R'],
 ['腹胀', 15, 16, 'R'],
 ['排便', 23, 24, 'R'],
 ['排气', 30, 31, 'R']]

In [36]:
cct.getLearnedEntities('LearnedETag')

[['腹痛', 12, 13, 'Sy'], ['腹胀', 15, 16, 'Sy']]

# 8. CorpAnnotation: `CorpAnnotation`

---


Many Functions are Added Here.

Such as `.CorpAnnotation()`, `.data()`, `.execute()`.

Most Important, CCT must use a Batch Dictionary to be initialized.

See the code of the new `.__init__()` below.

---


In [37]:
import pandas as pd

def getLearnedETag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None

class ChineseClinicalText(object):
    
    def __init__(self, batch, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        self.batch = batch
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
    

    def updateDF(self):
        #update the attribute for pandas PF form sentence
        DF = pd.DataFrame([i.to_dict() for i in self.atoms])
        cols = ['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags
        self.dfFormat = DF[cols] # Here is to have a good order
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


       
    ############ Deal With Learned E Tags, P1 ###########
    def corpResult(self, result, newTag):
        assert newTag == 'LearnedETag'
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getLearnedETag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
        self.updateDF()
                     
         
    ############ Deal With Learned E Tags, P2 ###########
    def getLearnedEntities(self, eType):
        ''' eType could only be 'LearnedETag' '''
        assert eType == 'LearnedETag'
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        self.learnedEntities = [tuple(i) for i in entitiesList]
        return entitiesList


    ############ NEW: Deal With Annotated R and E Tags #####
    def getAnnotedEntities(self, eType, annotedFilePath = None):
        assert eType == 'ETag' or eType == 'RTag'
        fLabel = self.batch['dataAnno']['fLabel']
        sep =  self.batch['dataAnno']['sep']
        start = self.batch['dataAnno']['start']
        if annotedFilePath == None:
            annotedFilePath = self.annotedFilePath
        annotedEntities = []
        with open(annotedFilePath, 'r') as f:
            for l in f.readlines():
                entity = strQ2B(l).replace('\n','').split(sep)
                entity[-1] = fLabel[entity[-1]]
                entity[1], entity[2] = int(entity[1])-start, int(entity[2])-start
                annotedEntities.append(entity)
        if eType == 'RTag':
            for ae in annotedEntities:
                ae[-1] = 'R'
            self.annotedREntities = [tuple(i) for i in annotedEntities]
        
        annotedEntities = [tuple(i) for i in annotedEntities]
        self.annotedEntities = annotedEntities
        return annotedEntities
        
    def corpAnnotation(self, newTag):
        assert newTag == 'ETag' or newTag == 'RTag'
        entityTagsList = []
        if newTag == 'RTag':
            for ae in self.annotedREntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        elif newTag == 'ETag':
            for ae in self.annotedEntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        for a in self.atoms:
            a.set_attr(newTag, 'O')
            for eTag in entityTagsList:
                if a.index == eTag[0] and a._atom == eTag[1]:
                    a.set_attr(newTag, eTag[-1])
    
        self.newTags.append(newTag)
        self.updateDF()
        
        
    ############  Other Tools to Faciliate the Processes #########
    def data(self):
        s = {}
        # Num of Atoms
        s['Atoms'] = len(self.atoms)
        s['E_Learned'] = len(self.learnedEntities)
        s['E_Annoted'] = len(self.annotedEntities)
        s['E_Matched'] = len(set(self.learnedEntities).intersection(set(self.annotedEntities)))## Union vs Join
        
        entitiesLable = list(self.batch['dataAnno']['fLabel'].values())
        for eL in entitiesLable:
            elL = [e for e in self.learnedEntities if eL == e[-1]]
            elA = [e for e in self.annotedEntities if eL == e[-1]]
            elM = set(elA).intersection(set(elL)) ## Union vs Join
            s[eL+'_Learned'] = len(elL)
            s[eL+'_Annoted'] = len(elA)
            s[eL+'_Matched'] = len(elM)
        
        return s
    
    def execute(self):
        self.setAtomTags()   # set atoms' tags
        self.getPOSTags()    # generate atoms' POS tags. (only valid in text)
        self.setPOSTags()    # set atoms' POS tags
        self.updateDF()      # update atoms' attributes to DF format
    

batch1SampleOrig = 'demo/Text/batch1/orig.txt'
batch1SampleAnno = 'demo/Text/batch1/anno.txt'
batch2SampleOrig = 'demo/Text/batch2/orig.txt'
batch2SampleAnno = 'demo/Text/batch2/anno.txt'

In [38]:
print(batch1SampleAnno, batch1SampleOrig)

cct = ChineseClinicalText(batch1,
                          originalFilePath= batch1SampleOrig, 
                          annotedFilePath= batch1SampleAnno)
cct.execute()
print(cols)
Orig_Test_Path =  'demo/Text/batch1/orig_test.txt'
cct.toTextFile(Orig_Test_Path, cols = cols)
# Take as granted
TaggedResultPath = 'demo/Text/batch1/orig_tagged.txt'
result = pd.read_csv(TaggedResultPath, sep = '\t', comment = '#', 
                     header= None, skip_blank_lines= False)
result.head()

demo/Text/batch1/anno.txt demo/Text/batch1/orig.txt
['_atom', 'basicTag', 'dictTag', 'POSTag']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,中,CHN,OTHER,t-B,O/0.825244,Bo-B/0.000241,Bo-I/0.000977,Ch-B/0.000126,Ch-I/0.001453,DT-B/0.004608,DT-I/0.001729,O/0.825244,Si-B/0.148639,Si-I/0.013709,Sy-B/0.000620,Sy-I/0.000649,Tr-B/0.000644,Tr-I/0.000572,unct-B/0.000395,unct-I/0.000394
1,年,CHN,OTHER,t-I,O/0.837903,Bo-B/0.000004,Bo-I/0.000212,Ch-B/0.000003,Ch-I/0.000276,DT-B/0.000017,DT-I/0.001286,O/0.837903,Si-B/0.000130,Si-I/0.158759,Sy-B/0.000036,Sy-I/0.000867,Tr-B/0.000028,Tr-I/0.000333,unct-B/0.000011,unct-I/0.000134
2,男,CHN,OTHER,n-B,O/0.840708,Bo-B/0.000065,Bo-I/0.000079,Ch-B/0.000088,Ch-I/0.000065,DT-B/0.000008,DT-I/0.000569,O/0.840708,Si-B/0.000097,Si-I/0.156674,Sy-B/0.000208,Sy-I/0.000845,Tr-B/0.000096,Tr-I/0.000200,unct-B/0.000010,unct-I/0.000289
3,性,CHN,OTHER,n-I,O/0.845097,Bo-B/0.000003,Bo-I/0.000155,Ch-B/0.000007,Ch-I/0.000164,DT-B/0.000008,DT-I/0.000523,O/0.845097,Si-B/0.000002,Si-I/0.151787,Sy-B/0.000003,Sy-I/0.001120,Tr-B/0.000011,Tr-I/0.000256,unct-B/0.000004,unct-I/0.000860
4,",",PUNC,OTHER,x-B,O/0.999999,Bo-B/0.000000,Bo-I/0.000000,Ch-B/0.000000,Ch-I/0.000000,DT-B/0.000000,DT-I/0.000000,O/0.999999,Si-B/0.000000,Si-I/0.000001,Sy-B/0.000000,Sy-I/0.000000,Tr-B/0.000000,Tr-I/0.000000,unct-B/0.000000,unct-I/0.000000


In [39]:
cct.corpResult(result, 'LearnedETag')

cct.updateDF()
cct.dfFormat.head()

,_atom,basicTag,dictTag,radical,POSTag,LearnedETag
0,中,CHN,OTHER,丨,t-B,O
1,年,CHN,OTHER,干,t-I,O
2,男,CHN,OTHER,田,n-B,O
3,性,CHN,OTHER,忄,n-I,O
4,",",PUNC,OTHER,-,x-B,O


In [41]:
cct.getAnnotedEntities('ETag')
cct.getLearnedEntities('LearnedETag')

a = cct.annotedEntities
l = cct.learnedEntities
print(a)
print(l)
cct.data()

[('腹痛', 12, 13, 'Sy'), ('腹胀', 15, 16, 'Sy'), ('排便', 23, 24, 'Sy'), ('排气', 30, 31, 'Sy')]
[('腹痛', 12, 13, 'Sy'), ('腹胀', 15, 16, 'Sy')]


{'Atoms': 48,
 'Bo_Annoted': 0,
 'Bo_Learned': 0,
 'Bo_Matched': 0,
 'Ch_Annoted': 0,
 'Ch_Learned': 0,
 'Ch_Matched': 0,
 'E_Annoted': 4,
 'E_Learned': 2,
 'E_Matched': 2,
 'Si_Annoted': 0,
 'Si_Learned': 0,
 'Si_Matched': 0,
 'Sy_Annoted': 4,
 'Sy_Learned': 2,
 'Sy_Matched': 2,
 'Tr_Annoted': 0,
 'Tr_Learned': 0,
 'Tr_Matched': 0}

# 9. All in One

---- 

Test could be started from here

---

In [1]:
import sys

rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')

########################## text.py #######################

import pandas as pd
from jieba import posseg as ppseg
import string

from atom import Atom


def strQ2B(ustring):
    rstring = ''
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):
            inside_code -= 65248
        # 2: unichr; 3: chr
        rstring += chr(inside_code)
    return rstring

def cleanSpace(text):
    #return text.replace("\t"," ").replace(" ","&nbsp;").replace('\n', '')
    '''
    return text.strip()
    '''
    for i in ' \t\r\x0b\x0c':
        text = text.replace(i, '@')
    text = text.strip()
    text =  text.replace('\n', '@')
    return text 
    
def segmentStr(text):
    strLen = len(text)
    atoms = []
    for i in range(strLen):
        atoms.append(text[i])
    return atoms


def getPOSTagsList(text):
    segs = list(ppseg.cut(text))
    
    for i in range(len(segs)):
        pair = segs[i]
        start = sum(len(p.word) for p in segs[:i])
        end = sum(len(p.word) for p in segs[:i+1]) -1
        pair.indeces = [start, end]
        
    POSTagsList = []
    for p in segs:
        word = p.word
        for i in range(len(p.word)):
            if i == 0:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-B'])
            else:
                POSTagsList.append([p.indeces[0] + i, word[i], p.flag+'-I'])
                
    return POSTagsList


def getLearnedETag(data):
    if type(data) == str:
        return data.split('/')[0]
    else:
        return None
    

class ChineseClinicalText(object):
    
    def __init__(self, batch, text = None, originalFilePath = None, annotedFilePath = None):
        # to do: parse the text for this Chinise Clincal Text
        self.batch = batch
        if text != None:
            self._text = cleanSpace(strQ2B(text))
            
            self.atomsBaseList = segmentStr(self._text)
            self.newTags = []
            self.dfFormat = []
            self.score = None
            learnedREntities = []
            annotedREntities = []
        else:
            if originalFilePath != None:
                with open(originalFilePath, 'r') as f:
                    text = f.read() 
                self._text = cleanSpace(strQ2B(text))

                self.atomsBaseList = segmentStr(self._text)
                self.originalFilePath = originalFilePath
                self.newTags = []
                self.dfFormat = []
                self.score = None
                learnedREntities = []
                annotedREntities = []
                if annotedFilePath != None:
                    self.annotedFilePath = annotedFilePath
            else:
                print('Need a Text!!!')
                
    def setAtomTags(self):
        self.atoms = []
        for i in range(len(self.atomsBaseList)):
            a = Atom(self.atomsBaseList[i])
            a.index = i
            self.atoms.append(a)
            
    def getPOSTags(self):
        self.POSTags = getPOSTagsList(self._text)
        
    def setPOSTags(self):
        for a in self.atoms:
            a.POSTag = 'o'
            for pos in self.POSTags:
                if a.index == pos[0] and a._atom == pos[1]:
                    a.POSTag = pos[-1]
    

    def updateDF(self):
        #update the attribute for pandas PF form sentence
        DF = pd.DataFrame([i.to_dict() for i in self.atoms])
        cols = ['_atom', 'basicTag', 'dictTag', 'radical', 'POSTag'] + self.newTags
        self.dfFormat = DF[cols] # Here is to have a good order
        
    def toTextFile(self, filePath, cols = None):
        if cols != None:
            self.outputCols = cols
        self.dfFormat.to_csv(filePath, columns = self.outputCols,
                             sep = '\t', encoding = 'utf=8', 
                             header = False, index = False )


       
    ############ Deal With Learned E Tags, P1 ###########
    def corpResult(self, result, newTag):
        assert newTag == 'LearnedETag'
        result.columns = self.outputCols + ['Result'] + list(range((len(result.columns)-len(self.outputCols) -1)))
        result[newTag] = result['Result'].apply(getLearnedETag)
        result = result[self.outputCols + [newTag]]
        for idx, row in result.iterrows():
            for a in self.atoms:
                if a.index == idx and a._atom == row['_atom']:
                    a.set_attr(newTag, row[newTag])
        self.newTags.append(newTag)
        self.updateDF()
                     
         
    ############ Deal With Learned E Tags, P2 ###########
    def getLearnedEntities(self, eType):
        ''' eType could only be 'LearnedETag' '''
        assert eType == 'LearnedETag'
        taggedAtoms = [a for a in self.atoms if a.get_attr(eType) != 'O']
        startIdx = [idx for idx in range(len(taggedAtoms)) if taggedAtoms[idx].get_attr(eType)[-2:] == '-B']
        startIdx.append(len(taggedAtoms))
        entitiesList = []
        for i in range(len(startIdx)-1):
            entityAtom = taggedAtoms[startIdx[i]: startIdx[i+1]]
            startA, endA = entityAtom[0], entityAtom[-1]
            entitiesList.append([''.join(ea._atom for ea in entityAtom), startA.index, endA.index, startA.get_attr(eType).split('-')[0]])
        self.learnedEntities = [tuple(i) for i in entitiesList]
        return entitiesList


    ############ NEW: Deal With Annotated R and E Tags #####
    def getAnnotedEntities(self, eType, annotedFilePath = None):
        assert eType == 'ETag' or eType == 'RTag'
        fLabel = self.batch['dataAnno']['fLabel']
        sep =  self.batch['dataAnno']['sep']
        start = self.batch['dataAnno']['start']
        if annotedFilePath == None:
            annotedFilePath = self.annotedFilePath
        annotedEntities = []
        with open(annotedFilePath, 'r') as f:
            for l in f.readlines():
                entity = strQ2B(l).replace('\n','').split(sep)
                entity[-1] = fLabel[entity[-1]]
                entity[1], entity[2] = int(entity[1])-start, int(entity[2])-start
                annotedEntities.append(entity)
        if eType == 'RTag':
            for ae in annotedEntities:
                ae[-1] = 'R'
            self.annotedREntities = [tuple(i) for i in annotedEntities]
        
        annotedEntities = [tuple(i) for i in annotedEntities]
        self.annotedEntities = annotedEntities
        return annotedEntities
        
    def corpAnnotation(self, newTag):
        assert newTag == 'ETag' or newTag == 'RTag'
        entityTagsList = []
        if newTag == 'RTag':
            for ae in self.annotedREntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        elif newTag == 'ETag':
            for ae in self.annotedEntities:
                word = ae[0]
                for i in range(len(word)):
                    if i == 0:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-B'])
                    else:
                        entityTagsList.append([ae[1]+i, word[i], ae[-1] +'-I'])
                        
        for a in self.atoms:
            a.set_attr(newTag, 'O')
            for eTag in entityTagsList:
                if a.index == eTag[0] and a._atom == eTag[1]:
                    a.set_attr(newTag, eTag[-1])
    
        self.newTags.append(newTag)
        self.updateDF()
        
        
    ############  Other Tools to Faciliate the Processes #########
    def data(self):
        s = {}
        # Num of Atoms
        s['Atoms'] = len(self.atoms)
        s['E_Learned'] = len(self.learnedEntities)
        s['E_Annoted'] = len(self.annotedEntities)
        s['E_Matched'] = len(set(self.learnedEntities).intersection(set(self.annotedEntities)))## Union vs Join
        
        entitiesLable = list(self.batch['dataAnno']['fLabel'].values())
        for eL in entitiesLable:
            elL = [e for e in self.learnedEntities if eL == e[-1]]
            elA = [e for e in self.annotedEntities if eL == e[-1]]
            elM = set(elA).intersection(set(elL)) ## Union vs Join
            s[eL+'_Learned'] = len(elL)
            s[eL+'_Annoted'] = len(elA)
            s[eL+'_Matched'] = len(elM)
        
        return s
    
    def execute(self):
        self.setAtomTags()   # set atoms' tags
        self.getPOSTags()    # generate atoms' POS tags. (only valid in text)
        self.setPOSTags()    # set atoms' POS tags
        self.updateDF()      # update atoms' attributes to DF format
    
    

# 10. Final Test 

--- 

Prepare 

---

In [1]:
import sys

rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner'
sys.path.append('/Users/floyd/Desktop/Research/NER-CRF/cctner')

import pandas as pd

from dataset import batch1, batch2

### Batch1

----

Data From CCKS2017

---

In [2]:
batch1SampleOrig = 'demo/Text/batch1/orig.txt'
batch1SampleAnno = 'demo/Text/batch1/anno.txt'

batch1

{'dataAnno': {'fLabel': {'检查和检验': 'Ch',
   '治疗': 'Tr',
   '疾病和诊断': 'Si',
   '症状和体征': 'Sy',
   '身体部位': 'Bo'},
  'sep': '\t',
  'start': 0},
 'dataInput': {'anno_iden': '.txt',
  'filenames': ['一般项目', '病史特点', '诊疗经过', '出院情况'],
  'fpath': 'dataset/annoted/batch1/',
  'orig_iden': '.txtoriginal.txt',
  'rootPath': '/Users/floyd/Desktop/Research/NER-CRF/cctner/'},
 'name': 'CCKS'}

In [5]:
from text import ChineseClinicalText 

print(batch1SampleAnno)
print(batch1SampleOrig)

cct = ChineseClinicalText(batch1,
                          originalFilePath= batch1SampleOrig, 
                          annotedFilePath= batch1SampleAnno)


cct.execute()  

# This file is intend to be annoted by model

Orig_Test_Path = 'demo/Text/batch1/orig_test.txt'
cct.toTextFile(Orig_Test_Path, cols = ['_atom', 'basicTag', 'dictTag', 'POSTag'])

# Take as granted. This can be achieved by: 
# $crf_test -v 2 -m model-1abdp orig_test.txt > orig_tagged.txt
outputPath = 'demo/Text/batch1/orig_tagged.txt'
result = pd.read_csv(outputPath, sep = '\t', comment = '#', 
                     header= None, skip_blank_lines= False)


# all the annotated files can use these methods.
cct.getAnnotedEntities('RTag') # Get annotedREntities
cct.corpAnnotation('RTag')
cct.getAnnotedEntities('ETag') # Get annotedEntities
cct.corpAnnotation('ETag')

# Only the test data could used these methods
cct.corpResult(result, 'LearnedETag')
cct.getLearnedEntities('LearnedETag') # learnedEntities

cct.data()

demo/Text/batch1/anno.txt
demo/Text/batch1/orig.txt


{'Atoms': 48,
 'Bo_Annoted': 0,
 'Bo_Learned': 0,
 'Bo_Matched': 0,
 'Ch_Annoted': 0,
 'Ch_Learned': 0,
 'Ch_Matched': 0,
 'E_Annoted': 4,
 'E_Learned': 2,
 'E_Matched': 2,
 'Si_Annoted': 0,
 'Si_Learned': 0,
 'Si_Matched': 0,
 'Sy_Annoted': 4,
 'Sy_Learned': 2,
 'Sy_Matched': 2,
 'Tr_Annoted': 0,
 'Tr_Learned': 0,
 'Tr_Matched': 0}

In [6]:
cct.dfFormat.iloc[5:15]

,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag,LearnedETag
5,4,NUM,OTHER,-,m-B,O,O,O
6,9,NUM,OTHER,-,m-I,O,O,O
7,岁,CHN,OTHER,山,m-B,O,O,O
8,",",PUNC,OTHER,-,x-B,O,O,O
9,主,CHN,OTHER,丶,n-B,O,O,O
10,因,CHN,BREAK_UNIT,囗,n-I,O,O,O
11,:,PUNC,OTHER,-,x-B,O,O,O
12,腹,CHN,PART_UNIT,月,n-B,R-B,Sy-B,Sy-B
13,痛,CHN,SYM_UNIT,疒,n-I,R-I,Sy-I,Sy-I
14,、,PUNC,OTHER,-,x-B,O,O,O


In [7]:
cct.annotedEntities

[('腹痛', 12, 13, 'Sy'),
 ('腹胀', 15, 16, 'Sy'),
 ('排便', 23, 24, 'Sy'),
 ('排气', 30, 31, 'Sy')]

In [8]:
cct.learnedEntities

[('腹痛', 12, 13, 'Sy'), ('腹胀', 15, 16, 'Sy')]

### Batch2

----

Data From Luohu Hospital. Annotated by Researchers from SRIBD.

---

In [9]:
batch2SampleOrig = 'demo/Text/batch2/orig.txt'
batch2SampleAnno = 'demo/Text/batch2/anno.txt'

batch2

{'dataAnno': {'fLabel': {'不确定': 'unct',
   '检查项目': 'Ch',
   '治疗手段': 'Tr',
   '疾病名称': 'Si',
   '疾病类型': 'DT',
   '症状': 'Sy',
   '身体部位以及器官': 'Bo'},
  'sep': '\t',
  'start': 1},
 'dataInput': {'anno_iden': '_StandardFormat.txt',
  'filenames': ['text'],
  'fpath': 'dataset/annoted/batch2/',
  'orig_iden': '.txt',
  'rootPath': '/Users/floyd/Desktop/Research/NER-CRF/cctner/'},
 'name': 'LUOHU'}

In [10]:
# from text import ChineseClinicalText as CCT

print(batch2SampleAnno)
print(batch2SampleOrig)

cct = ChineseClinicalText(batch2,
                          originalFilePath= batch2SampleOrig, 
                          annotedFilePath= batch2SampleAnno)


cct.execute()  

# This file is intend to be annoted by model

Orig_Test_Path = 'demo/Text/batch2/orig_test.txt'
cct.toTextFile(Orig_Test_Path, cols = ['_atom', 'basicTag', 'dictTag', 'POSTag'])


# Take as granted. This can be achieved by: 
# $crf_test -v 2 -m model-1abdp orig_test.txt > orig_tagged.txt
outputPath = 'demo/Text/batch2/orig_tagged.txt'
result = pd.read_csv(outputPath, sep = '\t', comment = '#', 
                  header= None, skip_blank_lines= False)


cct.corpResult(result, 'LearnedETag')

cct.updateDF()
batch = batch2
cct.getAnnotedEntities('ETag')
cct.getLearnedEntities('LearnedETag')
cct.data()

demo/Text/batch2/anno.txt
demo/Text/batch2/orig.txt


{'Atoms': 345,
 'Bo_Annoted': 3,
 'Bo_Learned': 4,
 'Bo_Matched': 3,
 'Ch_Annoted': 1,
 'Ch_Learned': 1,
 'Ch_Matched': 1,
 'DT_Annoted': 1,
 'DT_Learned': 1,
 'DT_Matched': 1,
 'E_Annoted': 27,
 'E_Learned': 28,
 'E_Matched': 27,
 'Si_Annoted': 7,
 'Si_Learned': 7,
 'Si_Matched': 7,
 'Sy_Annoted': 14,
 'Sy_Learned': 14,
 'Sy_Matched': 14,
 'Tr_Annoted': 1,
 'Tr_Learned': 1,
 'Tr_Matched': 1,
 'unct_Annoted': 0,
 'unct_Learned': 0,
 'unct_Matched': 0}

In [11]:
print(len(cct.learnedEntities))
cct.learnedEntities

28


[('原发性胆汁性肝硬化', 0, 8, 'Si'),
 ('2级高血压病', 10, 15, 'Si'),
 ('中危', 17, 18, 'DT'),
 ('高脂血症', 21, 24, 'Si'),
 ('表皮样囊肿', 26, 30, 'Si'),
 ('皮下脂肪瘤', 56, 60, 'Si'),
 ('高血压病2级', 69, 74, 'Si'),
 ('高脂血症', 76, 79, 'Si'),
 ('降压药物治', 86, 90, 'Tr'),
 ('右上腹部不适', 107, 112, 'Sy'),
 ('贫血貌', 183, 185, 'Sy'),
 ('全身皮肤', 187, 190, 'Bo'),
 ('黄染', 192, 193, 'Sy'),
 ('出血点', 196, 198, 'Sy'),
 ('肝掌', 202, 203, 'Sy'),
 ('蜘蛛痣', 207, 209, 'Sy'),
 ('病理性杂音', 218, 222, 'Sy'),
 ('双肺', 224, 225, 'Bo'),
 ('干湿啰音', 234, 237, 'Sy'),
 ('腹壁静脉曲张', 244, 249, 'Sy'),
 ('Cullen征', 253, 259, 'Sy'),
 ('Grey-Turner征', 261, 272, 'Sy'),
 ('全腹', 277, 278, 'Bo'),
 ('压痛', 280, 281, 'Sy'),
 ('反跳痛', 283, 285, 'Sy'),
 ('肠鸣音', 319, 321, 'Ch'),
 ('双下肢', 327, 329, 'Bo'),
 ('水肿', 331, 332, 'Sy')]

In [12]:
print(len(cct.annotedEntities))
cct.annotedEntities

27


[('原发性胆汁性肝硬化', 0, 8, 'Si'),
 ('2级高血压病', 10, 15, 'Si'),
 ('中危', 17, 18, 'DT'),
 ('高脂血症', 21, 24, 'Si'),
 ('表皮样囊肿', 26, 30, 'Si'),
 ('皮下脂肪瘤', 56, 60, 'Si'),
 ('高血压病2级', 69, 74, 'Si'),
 ('高脂血症', 76, 79, 'Si'),
 ('降压药物治', 86, 90, 'Tr'),
 ('右上腹部不适', 107, 112, 'Sy'),
 ('贫血貌', 183, 185, 'Sy'),
 ('全身皮肤', 187, 190, 'Bo'),
 ('黄染', 192, 193, 'Sy'),
 ('出血点', 196, 198, 'Sy'),
 ('肝掌', 202, 203, 'Sy'),
 ('蜘蛛痣', 207, 209, 'Sy'),
 ('病理性杂音', 218, 222, 'Sy'),
 ('干湿啰音', 234, 237, 'Sy'),
 ('腹壁静脉曲张', 244, 249, 'Sy'),
 ('Cullen征', 253, 259, 'Sy'),
 ('Grey-Turner征', 261, 272, 'Sy'),
 ('全腹', 277, 278, 'Bo'),
 ('压痛', 280, 281, 'Sy'),
 ('反跳痛', 283, 285, 'Sy'),
 ('肠鸣音', 319, 321, 'Ch'),
 ('双下肢', 327, 329, 'Bo'),
 ('水肿', 331, 332, 'Sy')]